

# import pretrained sentiment model



In [ ]:



from transformers import pipeline
classifier = pipeline(
                      task="zero-shot-classification",
#                       device=0,
                      model="facebook/bart-large-mnli"
                    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import warnings
import pandas as pd

# test it on some phrase


In [ ]:

classifier("How Texans' D has evolved under Crennel Vrabel",["positive","negative",'neutral'],multi_label=True)



{'sequence': "How Texans' D has evolved under Crennel Vrabel",
 'labels': ['positive', 'negative', 'neutral'],
 'scores': [0.8774796724319458, 0.0286089014261961, 0.003866886720061302]}

# next 3 blocks are building a function to get the total sentiment score


In [ ]:
output = classifier("How Texans' D has evolved under Crennel Vrabel",["positive","negative",'neutral'],multi_class=True)


The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.


In [ ]:
positive = output['scores'][output['labels'].index('positive')]
negative = output['scores'][output['labels'].index('negative')]
total= positive - negative
print("Positive Score:", positive)

Positive Score: 0.8774796724319458


In [ ]:

def get_total_score(text):
    labels = ["positive", "negative", "neutral"]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        output = classifier(text, labels, multi_label=True)
    positive_score = output['scores'][output['labels'].index('positive')]
    negative_score = output['scores'][output['labels'].index('negative')]
    neutral_score = output['scores'][output['labels'].index('neutral')]

    if (neutral_score)>0.50:
        total_score = 0
    else:
        total_score = positive_score - negative_score


    return total_score

In [ ]:
# Example usage
text = "How Texans' D has evolved under Crennel Vrabel  "
total_score = get_total_score(text)
print("Total Score:", total_score)

Total Score: 0.8459009528160095


# Read and then clean up the data from the headlines

In [ ]:
df = pd.read_csv("/content/sorted_df_w_teams_andDates3.csv")



# df.info()

In [ ]:

# Drop duplicates from the "headlines" column
df.drop_duplicates(subset="headlines", inplace=True)


df = df.dropna()

# # Display the resulting dataframe
# df.info()

In [ ]:
df['headlines'] = df['headlines'].str.strip()


df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5378 entries, 19 to 7045
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   archiveDate  5378 non-null   object
 1   Urls         5378 non-null   object
 2   headlines    5378 non-null   object
 3   team         5378 non-null   object
dtypes: object(4)
memory usage: 210.1+ KB


In [ ]:
# prompt: select 101-200 of df and call it df2

df2 = df.iloc[4700:5378]


# This is where I apply it to the data. But it takes a lot of computing power and is slow so I just did 10 right here. We will probably have to do it in chunks if we actually use this

In [ ]:
df2['total_score'] = df2['headlines'].apply(lambda x: get_total_score(x))


<ipython-input-50-7c4a206284f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['total_score'] = df2['headlines'].apply(lambda x: get_total_score(x))


In [ ]:
df2.head(100)

,archiveDate,Urls,headlines,team,total_score
6290,2021-08-22,http://web.archive.org/web/20210822002058/http...,In his first 2021 preseason start Steelers QB ...,Pittsburgh Steelers,0.940955
6291,2021-08-23,http://web.archive.org/web/20210823023637/http...,49ers QB Trey Lance has thrown for two touchdo...,San Francisco 49ers,0.931278
6292,2021-08-24,http://web.archive.org/web/20210824013328/http...,Can Trevor Lawrence and the Jaguars trim the S...,Jacksonville Jaguars,0.487247
6293,2021-08-24,http://web.archive.org/web/20210824165213/http...,Source: Panthers WR Anderson reach extension,Carolina Panthers,0.974520
6294,2021-08-24,http://web.archive.org/web/20210824165213/http...,Jameis makes Saints QB1 case with huge quarter,New Orleans Saints,0.993448
...,...,...,...,...,...
6392,2021-09-13,http://web.archive.org/web/20210913021244/http...,49ers fear CB Verrett tears ACL; Mostert hurt too,San Francisco 49ers,-0.814025
6393,2021-09-13,http://web.archive.org/web/20210913021244/http...,What drops? Bengals' Chase has big NFL debut,Cincinnati Bengals,0.304863
6394,2021-09-13,http://web.archive.org/web/20210913021244/http...,Winston leads Saints to stunning rout of Packers,New Orleans Saints,0.983766
6395,2021-09-14,http://web.archive.org/web/20210914002348/http...,The Ravens flock to Las Vegas to try to stake ...,Baltimore Ravens,0.524963


In [ ]:
#  df_scored = pd.read_csv("/content/df_scored_5.csv")




In [ ]:
# prompt: add df1 to df2 so they are the same dataframe

import pandas as pd
df_scored = pd.concat([df_scored, df2], axis=0)


In [ ]:
df_scored.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5370 entries, 0 to 7045
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   2695 non-null   float64
 1   archiveDate  5370 non-null   object 
 2   Urls         5370 non-null   object 
 3   headlines    5370 non-null   object 
 4   team         5370 non-null   object 
 5   total_score  5370 non-null   float64
dtypes: float64(2), object(4)
memory usage: 293.7+ KB


In [ ]:
df_scored.head(100)

,Unnamed: 0,archiveDate,Urls,headlines,team,total_score
0,19.0,2017-08-20,http://web.archive.org/web/20170820000936/http...,Jags must make QB change; Cutler could have ca...,Jacksonville Jaguars,0.023380
1,20.0,2017-08-20,http://web.archive.org/web/20170820000936/http...,How Texans' D has evolved under Crennel Vrabel,Houston Texans,0.848871
2,21.0,2017-08-20,http://web.archive.org/web/20170820000936/http...,Cutler's debut proves he's in control of Dolph...,Miami Dolphins,0.996091
3,22.0,2017-08-20,http://web.archive.org/web/20170820000936/http...,Raiders' Sean Smith charged with felony assault,Las Vegas Raiders,-0.629718
4,23.0,2017-08-20,http://web.archive.org/web/20170820000936/http...,Making sense of the Patriots fantasy RBs,New England Patriots,0.089151
...,...,...,...,...,...,...
95,232.0,2017-10-04,http://web.archive.org/web/20171004001019/http...,2018 draft order and needs for every team: Bro...,Cleveland Browns,-0.031795
96,236.0,2017-10-05,http://web.archive.org/web/20171005013616/http...,Chiefs G Zach Fulton Celebrates Win With Pizza...,Kansas City Chiefs,0.946683
97,237.0,2017-10-05,http://web.archive.org/web/20171005013616/http...,Game facts: Patriots at Buccaneers (Buccaneers...,New England Patriots,0.149175
98,240.0,2017-10-06,http://web.archive.org/web/20171006000925/http...,Patriots TE Gronkowski ruled out vs. Bucs,New England Patriots,-0.853042


In [ ]:
# prompt: save dataframe as a csv

df_scored.to_csv('/content/df_scored_9.csv')
